In [163]:
import numpy as np
import pandas as pd
import sklearn
import nltk

In [178]:
data = pd.read_csv("combined_data.csv")
data

,sentiment,Text
0,sadness,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,happy,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
61454,fear,Melissa stared at her friend in dism
61455,happy,Successive state elections have seen the gover...
61456,fear,Vincent was irritated but not dismay
61457,happy,Kendall-Hume turned back to face the dismayed ...


In [165]:
data["sentiment"].value_counts()

fear        16241
happy       13508
sadness      9796
neutral      8960
love         4720
anger        4069
surprise     2639
relief       1526
Name: sentiment, dtype: int64

In [179]:
# Removing punctuation, URL, and tags
import re
data['Text'] = data['Text'].apply(lambda x: re.sub("(@[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)",'',x))

In [180]:
data.dropna(inplace=True)

In [168]:
print(data.shape)
data.head()

(61459, 2)


,sentiment,Text
0,sadness,i know i was listenin to bad habit earlier a...
1,sadness,Layin n bed with a headache ughhhhwaitin on y...
2,sadness,Funeral ceremonygloomy friday
3,happy,wants to hang out with friends SOON
4,neutral,We want to trade with someone who has Houston...


In [181]:
from nltk.stem import PorterStemmer

In [182]:
stemmer = SnowballStemmer("english") 
# Split string on each row into a list of words
data['Text'] = data.apply(lambda x : x['Text'].split(" "),axis=1)
# Apply the stemmer on each word
data['Text'] = data['Text'].apply(lambda x: [stemmer.stem(y) for y in x])
# Join list of words into a single string for each row
data['Text'] = data['Text'].apply(' '.join)   # join list of strings into one string for each row
data

,sentiment,Text
0,sadness,i know i was listenin to bad habit earlier a...
1,sadness,layin n bed with a headach ughhhhwaitin on yo...
2,sadness,funer ceremonygloomi friday
3,happy,want to hang out with friend soon
4,neutral,we want to trade with someon who has houston ...
...,...,...
61454,fear,melissa stare at her friend in dism
61455,happy,success state elect have seen the govern parti...
61456,fear,vincent was irrit but not dismay
61457,happy,kendallhum turn back to face the dismay coup


#### Split Train and Test sets:

In [183]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['Text'], 
                                                    data['sentiment'],test_size=0.20, 
                                                    random_state=1)

#### Word of Bags: Preprocessing

In [184]:
from sklearn.feature_extraction.text import CountVectorizer
# Instantiate the CountVectorizer method
count_vector = CountVectorizer(stop_words='english', max_features = 5000)

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. 
testing_data = count_vector.transform(X_test)

In [172]:
training_data.shape

(49167, 5000)

In [186]:
count_vector.get_feature_names()

['aaaah',
 'aah',
 'aaron',
 'ab',
 'abandon',
 'abil',
 'abit',
 'abl',
 'abov',
 'absenc',
 'absolut',
 'abt',
 'abus',
 'ac',
 'academ',
 'academi',
 'accent',
 'accept',
 'access',
 'accessori',
 'accid',
 'accident',
 'accompani',
 'accomplish',
 'accord',
 'account',
 'accus',
 'ace',
 'ach',
 'achiev',
 'acid',
 'ack',
 'acknowledg',
 'acoust',
 'acquaint',
 'acquir',
 'act',
 'action',
 'activ',
 'actor',
 'actress',
 'actual',
 'acut',
 'ad',
 'adam',
 'adapt',
 'add',
 'addict',
 'addit',
 'address',
 'adjust',
 'administr',
 'admir',
 'admit',
 'adopt',
 'ador',
 'adrenalin',
 'adult',
 'advanc',
 'advantag',
 'adventur',
 'advertis',
 'advic',
 'advis',
 'affair',
 'affect',
 'affection',
 'affirm',
 'afford',
 'afraid',
 'african',
 'afternoon',
 'afterward',
 'age',
 'agenc',
 'agent',
 'aggrav',
 'aggress',
 'agh',
 'agit',
 'agitat',
 'ago',
 'agre',
 'ah',
 'aha',
 'ahaha',
 'ahead',
 'ahh',
 'ahhh',
 'ahhhh',
 'ahhhhh',
 'ahhhhhh',
 'aid',
 'aiden',
 'aim',
 'ain',
 '

### Multinomial Naive Bayes

In [187]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes = MultinomialNB()
naive_bayes.fit(training_data,y_train)

MultinomialNB()

In [188]:
predictions = naive_bayes.predict(testing_data)

In [189]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: {}'.format(accuracy_score(y_test, predictions)))

Accuracy score: 0.3661731207289294


### Attempt to tune hyperparameters:

In [190]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
print(naive_bayes.get_params())
parameters = {
    'alpha': np.linspace(0.001,1.5,26),
    'fit_prior' : [True,False]
}


{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}


In [191]:
from sklearn.metrics import classification_report
scores = ["precision", "recall"]
for score in scores:
    clf = GridSearchCV(MultinomialNB(),parameters, scoring="%s_macro" % score)
    clf.fit(training_data,y_train)
#     clf.predict(testing_data)
    y_true, y_pred = y_test, clf.predict(testing_data)
    report_dict = classification_report(y_true, y_pred, output_dict=True)
    pd.DataFrame(report_dict)
print(clf.best_score_)
print(clf.best_params_)

0.32713832875564985
{'alpha': 0.3008, 'fit_prior': False}


In [192]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes = MultinomialNB(alpha = 0.54064,fit_prior = False )
naive_bayes.fit(training_data,y_train)
predictions = naive_bayes.predict(testing_data)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: {}'.format(accuracy_score(y_test, predictions)))

Accuracy score: 0.3128864301985031
